## Mapping Results
Answering the questions:
1. Where were the heat violations?
2. Where were sensor failures?

### Setup

In [3]:
import pandas as pd
import folium as fo

In [4]:
df = pd.read_csv("data/heatseek_geocoded.csv")

In [5]:
df.head()

,Unnamed: 0,user_id,address,zip_code,apartment,sensor_id,bbl,temp,created_at,outdoor_temp,violation,add_full,lat,lon
0,0,381.0,124 E 176th St,10453.0,6J,feather0090,2.028260e+09,66,2019-01-30 23:58:43,11.0,True,"124 E 176th St, New York City, New York, USA",40.847928,-73.908931
1,1,390.0,1802 Ocean Parkway,11223.0,C4,feather0101,3.066810e+09,69,2019-01-30 23:58:19,14.0,False,"1802 Ocean Parkway, New York City, New York, USA",40.604475,-73.967530
2,2,492.0,341 E 19th St,11226.0,3E,feather0118,3.051620e+09,68,2019-01-30 23:54:54,13.0,False,"341 E 19th St, New York City, New York, USA",40.735065,-73.981193
3,3,389.0,4836 41st St,11104.0,2F,feather0024,4.002020e+09,50,2019-01-30 23:50:40,16.0,True,"4836 41st St, New York City, New York, USA",40.738850,-73.924581
4,4,167.0,150 Court St,11201.0,NaN,feather0115,3.002920e+09,69,2019-01-30 23:50:16,14.0,False,"150 Court St, New York City, New York, USA",40.688979,-73.992963


In [6]:
n_records = df['user_id'].value_counts().reset_index()

### Mapping Heat Violations

In [7]:
# first summarize data such that each row = individual, create col `n_violations` which is total number of Trues
violations = df.groupby(['user_id', 'lat', 'lon', 'add_full'])[['violation']].sum().reset_index()
violations = pd.merge(violations, n_records, how = 'left', left_on = 'user_id', right_on = 'index')
violations['vio_perc'] = round(violations['violation'] / violations['user_id_y'] * 100, 2)

In [8]:
violations.head()

,user_id_x,lat,lon,add_full,violation,index,user_id_y,vio_perc
0,163.0,40.701940,-73.925959,"88 Starr St, New York City, New York, USA",135.0,163.0,5789,2.33
1,167.0,40.688979,-73.992963,"150 Court St, New York City, New York, USA",134.0,167.0,4245,3.16
2,381.0,40.847928,-73.908931,"124 E 176th St, New York City, New York, USA",456.0,381.0,3287,13.87
3,383.0,40.847928,-73.908931,"124 E 176th St, New York City, New York, USA",245.0,383.0,1489,16.45
4,385.0,40.678005,-73.999372,"68 4th Place, New York City, New York, USA",66.0,385.0,3370,1.96


In [9]:
# specifying basemap
violations_map = fo.Map(location = [40.7128, - 74.0060], tiles = 'OpenStreetMap', zoomstart = 15)

# create color dictionary for violations
violations['vio_q'] = pd.qcut(violations['vio_perc'], [0, 0.5, 0.75, 0.90, 1], labels = False)
vio_colors = {0: 'lightgrey', 1: 'lightgreen', 2: 'orange', 3: 'red'}

# plot points
for col, row in violations.iterrows():
    fo.CircleMarker(location = [row['lat'], row['lon']],
                    color = 'b',
                    fill = True,
                    fill_color = vio_colors[row['vio_q']],
                    fill_opacity = 0.7,
                    popup = ('<b>User ID</b>: ' + str(row['user_id_x']) + '<br>'
                             '<b>Address</b>: ' + str(row['add_full']) + '<br>'
                             '<b>Violation</b>: ' + str(row['vio_perc']) + '%' + '<br>'
                            )).add_to(violations_map)

# show map
violations_map

### Mapping Sensor Failures

In [10]:
sensor_failures = df.groupby(['user_id', 'lat', 'lon', 'add_full'])[['sensor_id']].nunique().reset_index()

In [17]:
# specifying basemap
sensors_map = fo.Map(location = [40.7128, - 74.0060], tiles = 'OpenStreetMap', zoomstart = 15)

# create color dictionary for failures
sensor_colors = {1: 'black', 2: 'red'}

# plot points
for col, row in sensor_failures.iterrows():
    fo.CircleMarker(location = [row['lat'], row['lon']],
                    color = 'b',
                    fill = True,
                    fill_color = sensor_colors[row['sensor_id']],
                    fill_opacity = 0.7,
                    popup = ('<b>User ID</b>: ' + str(row['user_id']) + '<br>'
                             '<b>Address</b>: ' + str(row['add_full']) + '<br>'
                             '<b>No. of Sensors</b>: ' + str(row['sensor_id']) + '<br>'
                            )).add_to(sensors_map)

# show map
sensors_map